In [ ]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append('..')
import d2lzh_pytorch as d2l

### 整体实现

In [ ]:
batch_size = 256
root='./Datasets/FashionMNIST'
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root, download=False)

num_epochs = 5
lr = 0.1

# 初始化模型参数
num_inputs = 784
num_outputs = 10
W = torch.tensor(np.random.normal(0,0.01,(num_inputs,num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

# 训练
for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in  train_iter:
            # 前向计算
            y_hat = net(X)
            # 计算loss
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            # 计算loss梯度
            l.backward()
            # 优化参数
            if optimizer is None:
                # 使用随机梯度进行优化
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()
                
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
            
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

# 预测
X, y = iter(test_iter).next()
true_labels = d2l.get_fashion_mnist_labels(y.numpy())
y_ = net(X).argmax(dim=1)
pred_labels = d2l.get_fashion_mnist_labels(y_.numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]
d2l.show_fashion_mnist(X[0:9], titles[0:9])

### 分步骤解析实现

#### 获取并读取数据

In [ ]:
batch_size = 256
root='./Datasets/FashionMNIST'
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root, download=False)

#### 初始化模型参数

In [ ]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0,0.01,(num_inputs,num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)
#需要模型参数梯度
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

#### 实现softmax运算

In [ ]:
def softmax(X):
    # X为batch_size行 num_outputs列的output
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True) # 每一行分别求和
    return X_exp / partition # 这里用了广播机制

#### 定义模型

In [ ]:
def net(X):
    # 通过view函数将每张原始图像改成长度为num_imputs的向量
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)

#### 定义损失函数

In [ ]:
def cross_entropy(y_hat, y):
    '''
    y_hat: batch_size个样本在num_outputs个类别上的预测概率
    y: 这batch_size个样本的数值标签
    交叉熵损失函数：H = -log(每个样本标签对应的预测概率)
    '''
    # gather函数可以获得每个样本标签对应的预测概率
    return - torch.log(y_hat.gather(1, y.view(-1,1)))

#### 计算分类准确率

In [ ]:
def accuracy(y_hat, y):
    '''
    y_hat: 预测概率分布
    y: 真是标签
    y_hat.argmax(dim=1)返回矩阵y_hat每行中最大元素的索引
    float()将前面的ByteTensor转化为值为0(假)或1(真)
    '''
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [ ]:
def evaluate_accuracy(data_iter, net):
    '''
    计算模型net在数据集data_iter上的准确率
    '''
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

#### 训练模型

In [ ]:
num_epochs = 5
lr = 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs,
              batch_size, params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in  train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            # 梯度清零
            
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                # 使用随机梯度进行优化
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()
                
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
            
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [ ]:
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

##### 预测

In [ ]:
X, y = iter(test_iter).next()
true_labels = d2l.get_fashion_mnist_labels(y.numpy())
y_ = net(X).argmax(dim=1)
pred_labels = d2l.get_fashion_mnist_labels(y_.numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]
d2l.show_fashion_mnist(X[0:9], titles[0:9])